In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline

In [19]:
# Load the dataset
file_path = '/content/Keywords Updated Data 01_08.xlsx'
data = pd.read_excel(file_path)

data.head()

,Keywords,Label
0,organic cotton,Environmental
1,recycled polyester,Environmental
2,natural dyes,Environmental
3,biodegradable materials,Environmental
4,sustainable fibers,Environmental


In [24]:
!pip -q install "numpy==1.23.5" "scipy==1.9.3" "gensim==4.3.2" "smart-open<7"


  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [26]:
import gensim, gensim.downloader as api

RuntimeError: Compiled extensions are unavailable. If you've installed from a package, ask the package maintainer to include compiled extensions. If you're building Gensim from source yourself, install Cython and a C compiler, and then run `python setup.py build_ext --inplace` to retry. 

In [ ]:
wv = api.load("word2vec-google-news-300")
wv.most_similar("innovation")[:5]


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import re

def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

# Custom transformer for converting text to Word2Vec embeddings using a pre-trained model
class Word2VecVectorizerPretrained(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
        self.dim = model.vector_size

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
        return np.array([
            np.mean([self.model[word] for word in simple_tokenizer(doc) if word in self.model]
                    or [np.zeros(self.dim)], axis=0)
            for doc in X
        ])

X = data['Keywords']
y = data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Creating a pipeline with Pre-trained Word2Vec Vectorizer and RandomForestClassifier
pipeline_pretrained = Pipeline([
    ('word2vec', Word2VecVectorizerPretrained(wv)),
    ('classifier', RandomForestClassifier(class_weight='balanced'))
])
# Training the model
pipeline_pretrained.fit(X_train, y_train)

# Predicting the test set
predictions = pipeline_pretrained.predict(X_test)
# Evaluating the best model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Output the results
print("Accuracy of Random Forest Model:", accuracy)
print("\nClassification Report:\n", report)


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
import re

# Custom basic tokenizer function
def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

# Custom transformer for converting text to Word2Vec embeddings using a pre-trained model
class Word2VecVectorizerPretrained(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
        self.dim = model.vector_size

    def fit(self, X, y=None):
        # No fitting needed, but the method must be defined
        return self

    def transform(self, X, y=None):
        return np.array([
            np.mean([self.model[word] for word in simple_tokenizer(doc) if word in self.model]
                    or [np.zeros(self.dim)], axis=0)
            for doc in X
        ])

# Data preparation
X = data['Keywords']
y = data['Label']

# Creating a pipeline with Pre-trained Word2Vec Vectorizer and RandomForestClassifier
pipeline_pretrained = Pipeline([
    ('word2vec', Word2VecVectorizerPretrained(wv)),
    ('classifier', RandomForestClassifier(class_weight='balanced'))
])

# Setting up K-Fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(pipeline_pretrained, X, y, cv=kfold)

# Output cross-validation results
print("K-Fold Cross-Validation Accuracy Scores:", cross_val_scores)
print("Mean K-Fold Cross-Validation Accuracy:", np.mean(cross_val_scores))

# Training the model on the entire dataset (optional for deployment purposes)
pipeline_pretrained.fit(X, y)

# Generating classification report based on cross-validation (average metrics)
predicted_y = cross_val_score(pipeline_pretrained, X, y, cv=kfold, scoring='accuracy')
print("\nCross-Validation Classification Report:\n")
print(f"Overall Accuracy: {np.mean(predicted_y):.2f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Making predictions with the pipeline_pretrained model
pretrained_model_predictions = pipeline_pretrained.predict(X_test)

# Get indices of the test set from y_test
test_indices = y_test.index

# Extracting the test keywords and their corresponding true labels
test_keywords = X_test.loc[test_indices]
test_true_labels = y_test

# Creating a DataFrame for visualization
visualization_df = pd.DataFrame({
    'Keywords': test_keywords,
    'True Label': test_true_labels,
    'Predicted Label': pretrained_model_predictions
})

# Visualization code (e.g., plotting a confusion matrix, etc.) goes here
# This is an example of how you might visualize the results
# You can tailor this to your specific needs
plt.figure(figsize=(10, 7))
sns.heatmap(pd.crosstab(visualization_df['True Label'], visualization_df['Predicted Label']), annot=True, fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix of Predictions')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Build confusion table
cm = pd.crosstab(visualization_df['True Label'],
                 visualization_df['Predicted Label'])

# Only scale fonts (not figure size)
sns.set_style("white")
sns.set_context("notebook", font_scale=1.6)   # titles/labels/ticks bigger

plt.figure(figsize=(10, 7))  # unchanged
ax = sns.heatmap(
    cm, annot=True, fmt="g", square=True,
    annot_kws={"size": 16, "weight": "bold"},  # cell numbers
    linewidths=.6, linecolor="white"
)

ax.set_xlabel("Predicted Label", fontsize=16, labelpad=10)
ax.set_ylabel("True Label", fontsize=16, labelpad=10)
ax.set_title("Confusion Matrix of Predictions", fontsize=20, pad=12)

# Ticks (keep horizontal like your example)
ax.tick_params(axis="both", labelsize=13)
plt.xticks(rotation=0)

# Colorbar tick size
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()


In [ ]:
from joblib import dump

# Save the pipeline_pretrained model to a file
dump(pipeline_pretrained, 'pretrained_word2vec_random_forest_model.joblib')


In [ ]:
#from google.colab import files
#files.download('/content/pretrained_word2vec_random_forest_model.joblib')


In [ ]:
from joblib import load

# Load the model from the file
pipeline_pretrained = load('pretrained_word2vec_random_forest_model.joblib')


In [ ]:
file_path = '/content/Topic 1 Keywords.xlsx'
file_path_two='/content/Topic 2 Keywords.xlsx'
file_path_three='/content/Topic 3 Keywords.xlsx'
file_path_four='/content/topic_4.xlsx'
import pandas as pd
# Load the Excel file
df = pd.read_excel(file_path,nrows=11)
df_two = pd.read_excel(file_path_two,nrows=11)
df_three = pd.read_excel(file_path_three,nrows=11)
df_four = pd.read_excel(file_path_four,nrows=11)

# Display the first few rows of the dataframe
print(df.head())
print()
print(df_two.head())
print()
print(df_three.head())
print()
print(df_four.head())

In [ ]:
X_new = df['Keywords']
# Use the model to make predictions on the new data
new_predictions = pipeline_pretrained.predict(X_new)

X_new_two = df_two['Keywords']
# Use the model to make predictions on the new data
new_predictions_two = pipeline_pretrained.predict(X_new_two)

X_new_three = df_three['Keywords']
# Use the model to make predictions on the new data
new_predictions_three = pipeline_pretrained.predict(X_new_three)

X_new_four = df_four['Keywords']
# Use the model to make predictions on the new data
new_predictions_four = pipeline_pretrained.predict(X_new_four)



In [ ]:
# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(new_predictions, columns=['Predictions'])
predictions_df_two = pd.DataFrame(new_predictions_two, columns=['Predictions'])
predictions_df_three = pd.DataFrame(new_predictions_three, columns=['Predictions'])
predictions_df_four = pd.DataFrame(new_predictions_four, columns=['Predictions'])

# Optionally, combine with the new data for a comprehensive view
result_df = df.copy()
result_df['Predictions'] = predictions_df

result_df_two = df_two.copy()
result_df_two['Predictions'] = predictions_df_two

result_df_three = df_three.copy()
result_df_three['Predictions'] = predictions_df_three

result_df_four = df_four.copy()
result_df_four['Predictions'] = predictions_df_four



# Save the predictions to a new Excel file
result_df.to_excel('/content/predictions1.xlsx', index=False)
result_df_two.to_excel('/content/predictions2.xlsx', index=False)
result_df_three.to_excel('/content/predictions3.xlsx', index=False)
result_df_four.to_excel('/content/predictions4.xlsx', index=False)


In [ ]:
print("Topic 1\n")
print(result_df.head(25))
print()
print("Topic 2\n")
print(result_df_two.head(25))
print()
print("Topic 3\n")
print(result_df_three.head(25))
print()
print("Topic 4\n")
print(result_df_four.head(25))

In [ ]:
# Frequency Distribution
frequency_distribution = predictions_df['Predictions'].value_counts()
print("Topic 1\n")
print(frequency_distribution)

frequency_distribution_two = predictions_df_two['Predictions'].value_counts()
print("\nTopic 2\n")
print(frequency_distribution_two)

frequency_distribution_three = predictions_df_three['Predictions'].value_counts()
print("\nTopic 3\n")
print(frequency_distribution_three)

frequency_distribution_four = predictions_df_four['Predictions'].value_counts()
print("\nTopic 4\n")
print(frequency_distribution_four)



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

try:
    import seaborn as sns
    sns.reset_orig()
except Exception:
    pass
plt.rcdefaults()

data = {
    'Topic 1': {'Social': 5, 'Environmental': 4, 'Governance': 2},
    'Topic 2': {'Environmental': 8, 'Governance': 3, 'Social': 0},
    'Topic 3': {'Social': 7, 'Environmental': 4, 'Governance': 0},
    'Topic 4': {'Governance': 5, 'Environmental': 3, 'Social': 3}
}

colors = {'Social': 'skyblue', 'Environmental': 'lightgreen', 'Governance': 'lightcoral'}

fig, axs = plt.subplots(2, 2, figsize=(5.6, 5.6))
axs = axs.flatten()

def custom_autopct(pct):
    return f'{pct:.1f}%' if pct > 0 else ''

for i, (topic, distribution) in enumerate(data.items()):
    distribution_colors = [colors[k] for k in distribution.keys()]
    axs[i].pie(
        distribution.values(),
        autopct=lambda pct: custom_autopct(pct),
        startangle=140,
        colors=distribution_colors,
        textprops={'fontsize': 11, 'weight': 'semibold'},
        wedgeprops={'linewidth': 1, 'edgecolor': 'black'}
    )
    maj = max(distribution, key=distribution.get)
    axs[i].set_title(f"{topic}\nLabel: {maj}", fontsize=12, weight='bold', pad=6)


fig.subplots_adjust(right=0.86, wspace=0.28, hspace=0.40)


labels = list(colors.keys())
handles = [Patch(facecolor=colors[l], edgecolor='black', label=l) for l in labels]
fig.legend(
    handles, labels,
    loc='center left',
    bbox_to_anchor=(0.88, 0.5),
    fontsize=11, frameon=True
)

plt.show()


In [ ]:
file_path = '/content/Company1_Keywords.xlsx'
df_company = pd.read_excel(file_path,nrows=11)
X_new = df_company['Keywords']
new_predictions_company = pipeline_pretrained.predict(X_new)


predictions_df_company = pd.DataFrame(new_predictions_company, columns=['Predictions'])

result_df_company = df_company.copy()
result_df_company['Predictions'] = predictions_df_company

result_df_company.to_excel('/content/predictions8.xlsx', index=False)
print("\nComany 1 Predictions")

print(result_df_company.head(11))
print("\nComany 1")

frequency_distribution_company = predictions_df_company['Predictions'].value_counts()
print(frequency_distribution_company)

In [ ]:
plt.figure(figsize=(5, 5))
frequency_distribution_company.plot.pie(autopct='%1.1f%%', startangle=140)
plt.title('Company 1 Prediction Distribution')
plt.ylabel('')